# Import packages

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler

import torchinfo
from torch_snippets import Report
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from TrainValidate import TrainValidate, create_weighted_sampler

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.has_mps:
    device = "mps"

In [3]:
device

'mps'

# Load datasets

In [14]:
features = pd.read_csv("../datasets/final_features_removed.csv")
labels = pd.read_csv("../datasets/final_labels_removed.csv")
sources = pd.read_csv("../datasets/final_sources_removed.csv")

# Create class for collecting data

In [16]:
class PriceDataset(Dataset):
    def __init__(self, features, labels):
        assert len(features) == len(labels)
        self.features = features
        self.labels = labels

    def __getitem__(self, item):
        features = self.features.to_numpy()[item]
        features = torch.tensor(features).float().to(device)

        labels = self.labels.to_numpy()[item]
        price = torch.tensor(labels[1]).float().to(device)

        return features, price

    def __len__(self):
        return len(self.features)

In [17]:
temp = PriceDataset(features, labels)
in_features = len(temp[0][0])

# Build the model

In [18]:
class PredictPrice(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.in_features = in_features

        self.hidden = nn.Sequential(nn.Linear(in_features, 128),
                                    nn.ReLU(),
                                    nn.Linear(128, 128),
                                    nn.ReLU(),
                                    nn.Linear(128, 256),
                                    nn.ReLU(),
                                    nn.Linear(256, 128),
                                    nn.ReLU())
        self.price = nn.Sequential(nn.Linear(128, 1),
                                   nn.ReLU())

    def forward(self, x):
        x = self.hidden(x)
        price = self.price(x)
        return price.squeeze()

In [19]:
model = PredictPrice(in_features)
torchinfo.summary(model, input_size=(1, in_features))

Layer (type:depth-idx)                   Output Shape              Param #
PredictPrice                             --                        --
├─Sequential: 1-1                        [1, 128]                  --
│    └─Linear: 2-1                       [1, 128]                  6,656
│    └─ReLU: 2-2                         [1, 128]                  --
│    └─Linear: 2-3                       [1, 128]                  16,512
│    └─ReLU: 2-4                         [1, 128]                  --
│    └─Linear: 2-5                       [1, 256]                  33,024
│    └─ReLU: 2-6                         [1, 256]                  --
│    └─Linear: 2-7                       [1, 128]                  32,896
│    └─ReLU: 2-8                         [1, 128]                  --
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-9                       [1, 1]                    129
│    └─ReLU: 2-10                        [1, 1]                    --

# Data standardization

In [20]:
scaler = StandardScaler()
scaler.fit(features)
features[:] = scaler.transform(features)

# Train the model
## Prepare data

In [21]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, random_state=1, test_size=0.1)

# Cross validation

In [22]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
torch.manual_seed(13)
in_features = len(x_train.iloc[0])
epochs = 1000

for fold, (train_id, val_id) in enumerate(kfold.split(x_train.index)):
    train_feature, train_label = x_train.iloc[train_id], y_train.iloc[train_id]
    val_feature, val_label = x_train.iloc[val_id], y_train.iloc[val_id]
    print("\n\n-------------This is fold {}----------------".format(fold))

    train_data = PriceDataset(train_feature, train_label)
    val_data = PriceDataset(val_feature, val_label)
    train_loader = DataLoader(train_data, batch_size=16, shuffle=False, drop_last=True)
    val_loader = DataLoader(val_data, batch_size=16, shuffle=False, drop_last=True)

    model = PredictPrice(in_features).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    train_validate = TrainValidate(model, nn.MSELoss(), optimizer)
    train_validate.set_loader(train_loader, val_loader)
    train_validate.train(epochs)

    train_validate.save_model("models/all_removed_full_epoch_1000_fold_{}.pth".format(fold))



-------------This is fold 0----------------
EPOCH: 0.795	train_loss: 2964534272.000	(5.32s - 6687.62s remaining))))

KeyboardInterrupt: 

# Testing the performance

In [36]:
filename = "models/all_removed_full_epoch_1000_fold_1.pth"

model = torch.load(filename, map_location=torch.device("cpu"))
model.eval()

prices = []
for i in range(len(x_test)):
    feature = torch.tensor((x_test.iloc[i])).float()
    pred_price = model(feature)
    prices.append(pred_price.detach().item())

pred = pd.DataFrame({"PredictPrice": prices, "Price": y_test["Price / Rent"]})

In [37]:
error = []
for i in range(len(pred)):
    truth = pred["Price"].iloc[i]
    predict = pred["PredictPrice"].iloc[i]
    error.append(abs(truth - predict) / truth)

avg_error = sum(error) / len(error)
error = pd.DataFrame({"Error": error})
error = error.rename(index={i: j for i, j in zip(error.index, pred.index)})
pred = pd.concat([pred, error], axis=1)

In [38]:
pred

,PredictPrice,Price,Error
3497,0.00000,525.0,1.000000
4669,310436.25000,270000.0,0.149764
5356,590802.37500,580000.0,0.018625
5835,0.00000,385.0,1.000000
8435,0.00000,1400.0,1.000000
...,...,...,...
7173,0.00000,1300.0,1.000000
1392,464215.90625,520000.0,0.107277
3639,295281.09375,450000.0,0.343820
7216,0.00000,750.0,1.000000


# Number of sale and rental data

In [13]:
features = pd.read_csv("../datasets/final_features.csv")
sale_rent = torch.tensor(features["Sale or Let"].values)
classes, count = sale_rent.unique(return_counts=True)

In [14]:
classes, count

(tensor([0, 1]), tensor([2959, 6218]))

# Weighted random sampler

In [ ]:
weight = 1.0 / count.float()
weight

In [ ]:
sample_weights = weight[sale_rent.squeeze().long()]

In [ ]:
sample_weights[:10]

In [ ]:
sale_rent[:10]

In [ ]:
generator = torch.Generator()

sampler = WeightedRandomSampler(weights=sample_weights,
                                num_samples=len(sample_weights),
                                generator=generator,
                                replacement=True)

In [ ]:
train_data = PriceDataset(features, labels)
train_loader = DataLoader(train_data, batch_size=2, sampler=sampler)

# Prepare data
## Load data

In [23]:
features = pd.read_csv("../datasets/final_features.csv")
labels = pd.read_csv("../datasets/final_labels.csv")
sources = pd.read_csv("../datasets/final_sources.csv")

## Data standardization

In [24]:
scaler = StandardScaler()
scaler.fit(features)
features[:] = scaler.transform(features)

## Split dataset

In [25]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, random_state=1, test_size=0.1)

## Cross validation

In [26]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
torch.manual_seed(13)
in_features = len(x_train.iloc[0])
epochs = 1000

for fold, (train_id, val_id) in enumerate(kfold.split(x_train.index)):
    train_feature, train_label = x_train.iloc[train_id], y_train.iloc[train_id]
    val_feature, val_label = x_train.iloc[val_id], y_train.iloc[val_id]
    print("\n\n-------------This is fold {}----------------".format(fold))

    train_data = PriceDataset(train_feature, train_label)
    val_data = PriceDataset(val_feature, val_label)

    train_sampler = create_weighted_sampler(train_feature["Sale or Let"].values)
    val_sampler = create_weighted_sampler(val_feature["Sale or Let"].values)

    train_loader = DataLoader(train_data, batch_size=16, drop_last=True, sampler=train_sampler)
    val_loader = DataLoader(val_data, batch_size=16, drop_last=True, sampler=val_sampler)

    model = PredictPrice(in_features).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    train_validate = TrainValidate(model, nn.MSELoss(), optimizer)
    train_validate.set_loader(train_loader, val_loader)
    train_validate.train(epochs)

    train_validate.save_model("models/all_removed_weighted_sampler_full_epoch_1000_fold_{}.pth".format(fold))



-------------This is fold 0----------------
EPOCH: 0.566	train_loss: 29949804544.000	(3.78s - 6685.29s remaining))

KeyboardInterrupt: 

# Test performance (weighted sampler)

In [27]:
filename = "models/all_removed_weighted_sampler_full_epoch_1000_fold_2.pth"

model = torch.load(filename, map_location=torch.device("cpu"))
model.eval()

prices = []
for i in range(len(x_test)):
    feature = torch.tensor((x_test.iloc[i])).float()
    pred_price = model(feature)
    prices.append(pred_price.detach().item())

pred = pd.DataFrame({"PredictPrice": prices, "Price": y_test["Price / Rent"]})

In [43]:
error = []
for i in range(len(pred)):
    truth = pred["Price"].iloc[i]
    predict = pred["PredictPrice"].iloc[i]
    error.append(abs(truth - predict) / truth)

avg_error = sum(error) / len(error)
error = pd.DataFrame({"Error": error})
error = error.rename(index={i: j for i, j in zip(error.index, pred.index)})
pred = pd.concat([pred, error], axis=1)

In [44]:
pred

,PredictPrice,Price,Error
3497,842.390564,525.0,0.604553
4669,348821.093750,270000.0,0.291930
5356,905768.000000,580000.0,0.561669
5835,551.455383,385.0,0.432352
8435,1037.775879,1400.0,0.258732
...,...,...,...
7173,868.048767,1300.0,0.332270
1392,522328.250000,520000.0,0.004477
3639,305598.312500,450000.0,0.320893
7216,1240.245605,750.0,0.653661


# Separate sale and rental data
## Load datasets

In [28]:
features = pd.read_csv("../datasets/final_features.csv")
labels = pd.read_csv("../datasets/final_labels.csv")
sources = pd.read_csv("../datasets/final_sources.csv")

## Split sale and rental data

In [29]:
sale_features = features[features["Sale or Let"] == 1]
sale_features = sale_features.loc[:, ~sale_features.columns.isin(["Sale or Let"])]
sale_labels = labels.iloc[sale_features.index]

rental_features = features[features["Sale or Let"] == 0]
rental_features = rental_features.loc[:, ~rental_features.columns.isin(["Sale or Let"])]
rental_labels = labels.iloc[rental_features.index]

## Data standardization

In [30]:
scaler = StandardScaler()

scaler.fit(sale_features)
scaled_sale_feature = sale_features.copy()
scaled_sale_feature[:] = scaler.fit_transform(scaled_sale_feature)

scaler.fit(rental_features)
scaled_rental_feature = rental_features.copy()
scaled_rental_feature[:] = scaler.fit_transform(scaled_rental_feature)

## Split sale data into training and validation

In [31]:
x_train, x_test, y_train, y_test = train_test_split(scaled_sale_feature, sale_labels, random_state=1, test_size=0.1)

## Train the model (sale)

In [32]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
torch.manual_seed(13)
in_features = len(x_train.iloc[0])
epochs = 1000

for fold, (train_id, val_id) in enumerate(kfold.split(x_train.index)):
    train_feature, train_label = x_train.iloc[train_id], y_train.iloc[train_id]
    val_feature, val_label = x_train.iloc[val_id], y_train.iloc[val_id]
    print("\n\n-------------This is fold {}----------------".format(fold))

    train_data = PriceDataset(train_feature, train_label)
    val_data = PriceDataset(val_feature, val_label)
    train_loader = DataLoader(train_data, batch_size=16, drop_last=True, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=16, drop_last=True, shuffle=True)

    model = PredictPrice(in_features).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    train_validate = TrainValidate(model, nn.MSELoss(), optimizer)
    train_validate.set_loader(train_loader, val_loader)
    train_validate.train(epochs)

    train_validate.save_model("models/sale_only_removed_full_epoch_1000_fold_{}.pth".format(fold))



-------------This is fold 0----------------
EPOCH: 0.713	train_loss: 18946342912.000	(3.26s - 4574.24s remaining)))

KeyboardInterrupt: 

## Test Performance (sale only)

In [19]:
filename = "models/sale_only_removed_full_epoch_1000_fold_1.pth"

model = torch.load(filename, map_location=torch.device("cpu"))
model.eval()

prices = []
for i in range(len(x_test)):
    feature = torch.tensor((x_test.iloc[i])).float()
    pred_price = model(feature)
    prices.append(pred_price.detach().item())

pred = pd.DataFrame({"PredictPrice": prices, "Price": y_test["Price / Rent"]})

In [20]:
error = []
for i in range(len(pred)):
    truth = pred["Price"].iloc[i]
    predict = pred["PredictPrice"].iloc[i]
    error.append(abs(truth - predict) / truth)

avg_error = sum(error) / len(error)
error = pd.DataFrame({"Error": error})
error = error.rename(index={i: j for i, j in zip(error.index, pred.index)})
pred = pd.concat([pred, error], axis=1)

In [21]:
pred

,PredictPrice,Price,Error
5071,187159.578125,300000.0,0.376135
6456,348365.812500,350000.0,0.004669
3547,241875.140625,200000.0,0.209376
6245,215270.140625,220000.0,0.021499
5482,557764.937500,460000.0,0.212532
...,...,...,...
7894,217496.171875,210000.0,0.035696
5417,423593.906250,235000.0,0.802527
6981,454386.437500,450000.0,0.009748
431,157286.703125,137000.0,0.148078


## Split rental data

In [33]:
x_train, x_test, y_train, y_test = train_test_split(scaled_rental_feature, rental_labels, random_state=1, test_size=0.1)

## Train the model (rental)

In [34]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
torch.manual_seed(13)
in_features = len(x_train.iloc[0])
epochs = 1000

for fold, (train_id, val_id) in enumerate(kfold.split(x_train.index)):
    train_feature, train_label = x_train.iloc[train_id], y_train.iloc[train_id]
    val_feature, val_label = x_train.iloc[val_id], y_train.iloc[val_id]
    print("\n\n-------------This is fold {}----------------".format(fold))

    train_data = PriceDataset(train_feature, train_label)
    val_data = PriceDataset(val_feature, val_label)
    train_loader = DataLoader(train_data, batch_size=16, drop_last=True, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=16, drop_last=True, shuffle=True)

    model = PredictPrice(in_features).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    train_validate = TrainValidate(model, nn.MSELoss(), optimizer)
    train_validate.set_loader(train_loader, val_loader)
    train_validate.train(epochs)

    train_validate.save_model("models/rental_only_removed_full_epoch_1000_fold_{}.pth".format(fold))



-------------This is fold 0----------------
EPOCH: 1.045	train_loss: 289251.406	(2.53s - 2415.45s remaining))

KeyboardInterrupt: 

## Test performance (rental only)

In [23]:
filename = "models/rental_only_removed_full_epoch_1000_fold_2.pth"

model = torch.load(filename, map_location=torch.device("cpu"))
model.eval()

prices = []
for i in range(len(x_test)):
    feature = torch.tensor((x_test.iloc[i])).float()
    pred_price = model(feature)
    prices.append(pred_price.detach().item())

pred = pd.DataFrame({"PredictPrice": prices, "Price": y_test["Price / Rent"]})

In [24]:
error = []
for i in range(len(pred)):
    truth = pred["Price"].iloc[i]
    predict = pred["PredictPrice"].iloc[i]
    error.append(abs(truth - predict) / truth)

avg_error = sum(error) / len(error)
error = pd.DataFrame({"Error": error})
error = error.rename(index={i: j for i, j in zip(error.index, pred.index)})
pred = pd.concat([pred, error], axis=1)

In [25]:
pred

,PredictPrice,Price,Error
2445,1172.183594,1400.0,0.162726
3121,0.000000,1.0,1.000000
3556,776.037659,750.0,0.034717
4569,733.201904,775.0,0.053933
4838,463.209137,475.0,0.024823
...,...,...,...
2812,1180.131714,1295.0,0.088701
803,1105.478638,850.0,0.300563
3567,649.455688,650.0,0.000837
8474,793.133179,675.0,0.175012
